In [1]:
import os, json
from datetime import datetime, timedelta, timezone
import pandas as pd

In [2]:
import plotly.express as px
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# read files

In [3]:
dirname = "C:/Users/david/singularity/midfreq/scrappers/defillama_history/data/DynYieldE"
pools = []
for filename in os.listdir(dirname):
    fullname = os.path.join(os.sep, dirname, filename)
    data = pd.read_csv(fullname, parse_dates=['date']).set_index('date')['haircut_apy']
    pools.append(data.rename(filename.replace('.csv','')))
history = pd.concat(pools, axis=1)

In [4]:
metadata = pd.read_csv("C:/Users/david/singularity/midfreq/scrappers/defillama_history/data/DynYieldE_pool_metadata.csv")
pool_map = metadata[['pool','project','symbol']].set_index('pool').T.to_dict()
pool_map = {key: '{}_{}'.format(x['project'], x['symbol']) for key, x in pool_map.items()}
index_map = {f'weight_{i}': f'w_{value}' for i, value in enumerate(pool_map.values())}
index_map |= {f'yield_{i}': f'y_{value}' for i, value in enumerate(pool_map.values())} 
pool_map

{'c8a24fee-ec00-4f38-86c0-9f6daebc4225': 'makerdao_DAI',
 'ccc467d1-bdf5-4062-81cd-7aed7dd9050b': 'makerdao_WSTETH',
 '51d2f8d4-1fb5-4f6b-938b-e9cd17ca1ceb': 'makerdao_WETH',
 '54600482-cfb9-4dd9-a73b-342cffd4644a': 'makerdao_WETH',
 '051c1a54-df05-4bc4-9d73-c17508acae23': 'makerdao_USDC',
 '51eb4b5c-6d62-42fe-99d6-92ba6eecd2d5': 'makerdao_WSTETH',
 '3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1': 'curve-dex_FRAX-USDC',
 'bd072651-d99c-4154-aeae-51f12109c054': 'convex-finance_FRAX-USDC',
 'efbf1a35-0756-41b1-b230-166ed4809314': 'makerdao_GUSD',
 '25171c4c-1877-449a-9f88-45a9f153ee31': 'curve-dex_DAI-USDC-USDT',
 '44c94cb2-8af3-4b7a-8cb5-0784ddcf3471': 'makerdao_WETH',
 'bd335b46-3daf-4507-adb2-6fd4337ea33b': 'makerdao_WBTC',
 'f38a1e55-ee6a-427c-83b5-74232e1b2c24': 'makerdao_G-UNI',
 '6ece81c6-2b1d-4513-ad1c-eaf9d52f67cb': 'makerdao_PAX',
 'cefa9bb8-c230-459a-a855-3b94e96acd8c': 'compound_USDC',
 '4dab1166-614a-429a-831f-4af70fd29c7d': 'makerdao_WBTC',
 'cc110152-36c2-4e10-9c12-c5b4eb662143': '

# compute historical pool rank

In [5]:
max_rank = 10
end = datetime.now().replace(tzinfo=timezone.utc)
start = end - timedelta(days = 900)
df = history.fillna(method='ffill')
df = df[(df.index >= start)&(df.index <= end)]
best_only = df[df.rank(axis=1, ascending=False)<max_rank].dropna(how='all', axis=1)
#best_only = best_only.rename(columns=pool_map)
best_only.iplot()

In [6]:
# with open(os.path.join(os.sep, dirname, f'EverTop{max_rank}.json'), 'w') as fp:
#     json.dump(list(best_only.columns), fp=fp)

In [7]:
short_listed_protocols = [
            'compound',
            'morpho-compound'
            'compound-v3',
            'morpho-aave',
            'convex-finance',
            'uniswap-v3',
            'uniswap-v2',
            'aave-v2',
            'aave-v3',
            'curve-dex',
            'spark',
'makerdao']
for it in best_only.columns:
    descr = pool_map[it] if it in pool_map else []
    if any(protocol in descr for protocol in short_listed_protocols):
        print(f'- \"{it}\" # {descr}')

- "1343a280-7812-4bc3-8f98-d1c37e11d271" # morpho-aave_USDT
- "25171c4c-1877-449a-9f88-45a9f153ee31" # curve-dex_DAI-USDC-USDT
- "30339b96-be67-45e5-87cb-adcd91049803" # convex-finance_ADAI-AUSDC-AUSDT
- "325ad2d6-70b1-48d7-a557-c2c99a036f87" # morpho-aave_USDC
- "3665ee7e-6c5d-49d9-abb7-c47ab5d9d4ac" # aave-v3_DAI
- "3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1" # curve-dex_FRAX-USDC
- "405d8dad-5c99-4c91-90d3-82813ade1ff1" # aave-v2_DAI
- "57647093-2868-4e65-97ab-9cae8ec74e7d" # compound_USDT
- "60d657c9-5f63-4771-a85b-2cf8d507ec00" # aave-v2_USDT
- "7394f1bc-840a-4ff0-9e87-5e0ef932943a" # convex-finance_DAI-USDC-USDT
- "802d35a2-5bae-456f-a2a9-4e1b48e0dec5" # uniswap-v3_DAI-USDT
- "8ec9e3e2-1eb5-4c47-82de-edcc825ca4cc" # curve-dex_CDAI-CUSDC
- "a349fea4-d780-4e16-973e-70ca9b606db2" # aave-v2_USDC
- "a86ee795-54d9-4812-9148-b312967cefe5" # uniswap-v2_DAI-USDC
- "aa70268e-4b52-42bf-a116-608b370f9501" # aave-v3_USDC
- "b8bcdf8e-96ed-40ca-a7aa-aa048b9874e5" # morpho-aave_DAI
- "bd072651-d99c-41